In [1]:
import sys
import numba
import swifter
import timeit
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')
pd.set_option('display.max_colwidth', -1)
pd.set_option('display.float_format', lambda x: '%.3f' % x)
sns.set_style('whitegrid')
plt.style.use('seaborn-whitegrid')
%matplotlib inline

__author__ = 'HK Dambanemuya'
__version__ = 'Python 2'

'''
    Analysis originaly performed in Python 2 (deprecated)
    Seaborn, Statsmodel, and * imports broken in Python 3
'''

In [2]:
bid_1 = pd.read_csv("../Data/lender_bidamount_fe/lender_bidamount_fe_10k.csv")
bid_2 = pd.read_csv("../Data/lender_bidamount_fe/lender_bidamount_fe_20k.csv")
bid_3 = pd.read_csv("../Data/lender_bidamount_fe/lender_bidamount_fe_30k.csv")
bid_4 = pd.read_csv("../Data/lender_bidamount_fe/lender_bidamount_fe_40k.csv")
bid_5 = pd.read_csv("../Data/lender_bidamount_fe/lender_bidamount_fe_50k.csv")
bid_6 = pd.read_csv("../Data/lender_bidamount_fe/lender_bidamount_fe_60k.csv")
bid_7 = pd.read_csv("../Data/lender_bidamount_fe/lender_bidamount_fe_65k.csv")

In [3]:
bid_frames = [bid_1, bid_2, bid_3, bid_4, bid_5, bid_6, bid_7]

In [4]:
data = pd.concat(bid_frames)
data['Listing_Key'] = data.ListingKey
data = data[['Listing_Key', 'ListingKey', 'MemberKey', 'BidAmount', 'LenderMedianBidAmount', 'LenderMeanBidAmount', 'LenderMaxBidAmount', 'LenderSumBidAmount']]
data['LenderUnlikelyBid'] = (data.BidAmount > data.LenderMedianBidAmount).astype(int)
data['LenderSumUnlikelyBidAmount'] = (data.LenderUnlikelyBid *data.BidAmount)
data.head()

,Listing_Key,ListingKey,MemberKey,BidAmount,LenderMedianBidAmount,LenderMeanBidAmount,LenderMaxBidAmount,LenderSumBidAmount,LenderUnlikelyBid,LenderSumUnlikelyBidAmount
0,3FF03364988124297E13A05,3FF03364988124297E13A05,033E336453973913915CD6B,50.000,100.000,91.920,200.000,11306.210,0,0.000
1,25713364434040193913F01,25713364434040193913F01,033E336453973913915CD6B,50.000,100.000,87.726,200.000,12106.210,0,0.000
2,B8953365111581953FDF6FF,B8953365111581953FDF6FF,033E336453973913915CD6B,90.180,100.000,92.895,200.000,10775.850,0,0.000
3,5D64336458879460900B458,5D64336458879460900B458,033E336453973913915CD6B,50.000,100.000,93.052,200.000,7723.330,0,0.000
4,113D3366128622300E1DBFD,113D3366128622300E1DBFD,033E336453973913915CD6B,50.000,100.000,92.540,200.000,7773.330,0,0.000


In [5]:
del bid_1
del bid_2
del bid_3
del bid_4
del bid_5
del bid_6
del bid_7
del bid_frames

## Mean

In [6]:
# f_avg = {'ListingKey': ['max'],
#          'LenderTotalBids': ['mean'],
#          'LenderWinningBids': ['mean'],
#          'LenderBidWinningRate': ['mean']
# }
# lender_mean_attr = pd.DataFrame(data.groupby(["Listing_Key"]).agg(f_avg).as_matrix())
# lender_mean_attr = lender_mean_attr.rename(index=str, columns={0: "LenderAvgWinningBids", 
#                                                              1: "LenderAvgTotalBids",
#                                                              2: "LenderAvgBidWinningRate",
#                                                              3: "ListingKey"})
# lender_mean_attr.head()

## Median

In [7]:
# f_median = {'ListingKey': ['max'],
#          'LenderTotalBids': ['median'],
#          'LenderWinningBids': ['median'],
#          'LenderBidWinningRate': ['median']
# }
# lender_median_attr = pd.DataFrame(data.groupby(["Listing_Key"]).agg(f_median).as_matrix())
# lender_median_attr = lender_median_attr.rename(index=str, columns={0: "LenderMedianWinningBids", 
#                                                                  1: "LenderMedianTotalBids",
#                                                                  2: "LenderMedianBidWinningRate",
#                                                                  3: "ListingKey"})
# lender_median_attr.head()

## Lender Unlikely Bids

In [8]:
# f_unlikely = {'ListingKey': ['max'],
#               'LenderUnlikelyBids': ['sum']
    
# }
# lender_unlikely_attr = pd.DataFrame(data[["Listing_Key", 'ListingKey', "MemberKey", "LenderUnlikelyBids"]].groupby(["Listing_Key"]).agg(f_unlikely)

## Standard Deviation

In [ ]:
f_std = {'ListingKey': ['max'],
         'LenderMedianBidAmount': ['std'],
         'LenderMeanBidAmount': ['std'],
         'LenderMaxBidAmount': ['std'],
         'LenderSumBidAmount': ['std'],
         'LenderUnlikelyBid': ['sum'],
         'LenderSumUnlikelyBidAmount': ['sum']
}

lender_std_attr = pd.DataFrame(data.groupby(["Listing_Key"]).agg(f_std).as_matrix())
lender_std_attr = lender_std_attr.rename(index=str, columns={0: "LenderSumUnlikelyBidAmount", 
                                                             1: "LenderStdMedianBidAmount",
                                                             2: "LenderSumUnlikelyBid",
                                                             3: "LenderStdSumBidAmount",
                                                             4: "ListingKey",
                                                             5: "LenderMeanBidAmount",
                                                             6: "LenderMaxBidAmount"})
lender_std_attr.head()

In [ ]:
# data[data["ListingKey"] == "00003383856420083050622"].groupby(["Listing_Key"]).agg(f_std)

## Merge Bid Feature DataFrames

In [ ]:
# Prepare final data
# final_lender_data = lender_mean_attr.merge(lender_median_attr, on="ListingKey")
# final_lender_data = final_lender_data.merge(lender_std_attr, on="ListingKey")
final_lender_data = lender_std_attr
final_lender_data.head()

## Import Class Variable

In [ ]:
listing_data = pd.read_csv("../Data/ProjectLevelData.txt", sep="|")
listing_data = listing_data.loc[(listing_data['RepaidOrNot']==True) | (listing_data['RepaidOrNot']==False)]
listing_data = listing_data[["ListingKey", "RepaidOrNot"]]
listing_data.head()

## Merge Features and Class Variable

In [ ]:
final_data = final_lender_data.merge(listing_data, on="ListingKey")
final_data['RepaidOrNot'] = final_data['RepaidOrNot'].astype(int)
final_data.head()

## Check Feature Correlation

In [ ]:
corr = final_data.swifter.apply(pd.to_numeric, errors='coerce').corr(method='pearson')
mask = np.zeros_like(corr, dtype=np.bool)
mask[np.triu_indices_from(mask)] = True
plt.figure(figsize=(12,12))
sns.heatmap(corr, 
        xticklabels=corr.columns,
        yticklabels=corr.columns,
        cmap=sns.color_palette("coolwarm_r"),
        mask = mask,
        linewidths=.5,
        vmin=-1,
        vmax=1,
        annot=True)
plt.title("Variable Correlation Heatmap")
plt.show()

## Save Data to CSV

In [ ]:
## Save data to csv file
# final_lender_data.to_csv("../Data/lender_listing_attr_filtered.csv", index=False)
final_lender_data.to_csv("../Data/lender_bidamount_attr.csv", index=False)